In [1]:
import pandas as pd
import datetime
path = f'../../../siads591 data/debris_all_for_gabbard/'

In [2]:
dfs = []
freq = '5D'
for y in range(1960,2022):
    df = pd.read_pickle(f'{path}{y}.pkl.gz', compression="gzip")
    dfs.append(df)
raw_df = pd.concat(dfs)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 76588309 entries, 1960-08-16 11:33:28.410048 to 2021-01-19 00:11:17.669760
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   NORAD_CAT_ID  uint32 
 1   PERIOD        float32
 2   APOAPSIS      float32
 3   PERIAPSIS     float32
dtypes: float32(3), uint32(1)
memory usage: 1.7 GB


In [3]:
df = raw_df.reset_index().groupby(['NORAD_CAT_ID', pd.Grouper(key='EPOCH', freq=freq)]).mean()
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15426156 entries, (51, Timestamp('1960-08-16 00:00:00', freq='5D')) to (47308, Timestamp('2021-01-18 00:00:00', freq='5D'))
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   PERIOD     float32
 1   APOAPSIS   float32
 2   PERIAPSIS  float32
dtypes: float32(3)
memory usage: 235.6 MB


In [4]:
raw_df

,NORAD_CAT_ID,PERIOD,APOAPSIS,PERIAPSIS
EPOCH,,,,
1960-08-16 11:33:28.410048,51,118.248001,1690.036987,1514.431030
1960-08-16 11:41:13.296480,53,118.405998,1697.227051,1521.487061
1960-08-24 16:39:55.734048,51,118.286003,1690.285034,1517.640015
1960-08-24 16:52:59.167775,53,118.390999,1695.915039,1521.487061
1960-09-18 07:20:01.794048,51,118.246002,1689.970947,1514.365967
...,...,...,...,...
2021-01-18 21:54:04.595616,29110,97.584999,644.786011,641.546997
2021-01-18 22:33:11.999808,40274,96.713997,1046.178955,156.429001
2021-01-18 22:41:27.647232,39357,98.055000,671.218018,660.156006


In [5]:
df = raw_df.reset_index().groupby(['NORAD_CAT_ID', pd.Grouper(key='EPOCH', freq=freq)]).mean()

In [6]:
df

PERIOD     APOAPSIS    PERIAPSIS
NORAD_CAT_ID EPOCH                                           
51           1960-08-16  118.248001  1690.036987  1514.431030
             1960-08-21  118.286003  1690.285034  1517.640015
             1960-09-15  118.246002  1689.970947  1514.365967
             1960-10-05  118.293999  1679.358032  1529.286987
             1960-10-30  118.240997  1689.490967  1514.365967
...                             ...          ...          ...
47247        2021-01-08  127.855362  3356.688232   701.038391
             2021-01-13  127.840866  3343.689941   712.754211
             2021-01-18  127.831001  3328.905029   726.661011
47308        2021-01-13   90.420624   390.575256   199.735748
             2021-01-18   90.336250   383.107758   198.890259

[15426156 rows x 3 columns]

In [7]:
all_dates = pd.date_range(start=df.index.get_level_values(1).min(), end=df.index.get_level_values(1).max(), freq=freq)

In [8]:
df2 = df.unstack(level=0).reindex(all_dates).interpolate(method='linear', limit_area="inside", limit=1, limit_direction='both').stack().reset_index().rename(columns={'level_0':'EPOCH'})

In [9]:
df2

,EPOCH,NORAD_CAT_ID,PERIOD,APOAPSIS,PERIAPSIS
0,1960-08-16,51,118.248001,1690.036987,1514.431030
1,1960-08-16,53,118.405998,1697.227051,1521.487061
2,1960-08-21,51,118.286003,1690.285034,1517.640015
3,1960-08-21,53,118.390999,1695.915039,1521.487061
4,1960-08-26,51,118.278000,1690.222168,1516.985229
...,...,...,...,...,...
16646236,2021-01-18,47224,97.203667,629.120667,620.567322
16646237,2021-01-18,47225,97.277000,632.122314,624.629028
16646238,2021-01-18,47246,123.740997,3085.439941,609.557007
16646239,2021-01-18,47247,127.831001,3328.905029,726.661011


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16646241 entries, 0 to 16646240
Data columns (total 5 columns):
 #   Column        Dtype         
---  ------        -----         
 0   EPOCH         datetime64[ns]
 1   NORAD_CAT_ID  uint64        
 2   PERIOD        float32       
 3   APOAPSIS      float32       
 4   PERIAPSIS     float32       
dtypes: datetime64[ns](1), float32(3), uint64(1)
memory usage: 444.5 MB


In [11]:
df2.to_pickle(f"./gabbard_out.pkl")